<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
import pandas as pd
import jax
from jax.lax import stop_gradient
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che
R=8.314 # J/(mol K)

In [3]:

p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r.Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
    W_n_vap = r.Pw * r.V_vap / (R * c.T)
    P = (c.air_n + W_n_vap) * R * c.T / r.V_vap
    # VSC is intended for solving models with multiple equations and expects
    # an iterable of (left,right) tuples
    return ( (s.P_f, P), )

s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(360,300,600)

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [4]:
# transform our model into one that takes a single array of adjustable variables
# for minimization.  Unnecessary for this simple model with just 1 variable, 
# but very helpful for more complex problems later.
vsc=VSC(v,s, model)
vsc.solve()

0.001973574785335787
0.0006606517658790154
0.0006606517658790154
1.5272366214779895e-05
1.5272366214779895e-05
1.5272366214779895e-05
1.213645372822225e-08
1.213645372822225e-08
1.213645372822225e-08
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.409037068602645e-20


In [5]:
# Dataframe of adjustable variables
vsc.vdf

,vector1
,1
T,365.719235


In [6]:
# Dataframe of intermediate results
vsc.rdf

,vector1
,1
Pw,77275.424480
V_vap,0.008995


In [7]:
# Dataframe of static state parameters
vsc.sdf

,vector1
,1
W_tot,1.000000
V_tot,0.010000
P_i,100000.000000
P_f,200000.000000
T_i,298.000000
Vw_i,0.001005
Vair_i,0.008995
air_n,0.363057


In [8]:
def model2(c,r):
    Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i 
    r.W_n_vap = Pw * r.V_vap / (R * c.T) 
    P = (s.air_n + r.W_n_vap) * R * c.T / r.V_vap

    # tuples from left and right side of equations
    P_constraint = (s.P_f, P)
    W_n_constraint = (r.W_n_vap, c.W_n_vap_desired)
    return (P_constraint, W_n_constraint)

In [9]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.
s.W_n_vap_desired = 0.3

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(v,s, model2)
vsc.solve()

0.10764694306583505
0.030440692345800494
0.030440692345800494
0.0012712658793009844
0.0012712658793009844
1.553728074595461e-05
1.553728074595461e-05
1.553728074595461e-05
2.9016926539896234e-09
2.9016926539896234e-09
2.9016926539896234e-09
3.1052252302777553e-14
3.1052252302777553e-14
3.1052252302777553e-14
3.1052252302777553e-14
3.1052252302777553e-14
8.924222365022703e-20


In [10]:
vsc.vdf

,vector1
,1
T,370.005638
V_tot,0.011204


In [11]:
vsc.rdf


,vector1
,1
V_vap,0.010199
W_n_vap,0.300000


In [12]:
def model3(c, r):
    Pw = p.Pvap(c.T)[3]
    # volume of liquid water lost as water vapor ignored in model1
    # evaluate the density of water at T, but do not propagate gradients through.
    r.v_w_vap = c.W_n_vap*p.Mw[3]/1000/p.rhol(stop_gradient(c.T))[3]
    V_vap = c.V_tot - (c.Vw_i - r.v_w_vap)
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (s.air_n + r.W_n_vap) * R * c.T / V_vap
    P_constraint = (s.P_f, P)
    W_constraint = (c.W_n_vap, r.W_n_vap)
    return (P_constraint,W_constraint)

In [13]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350., 300., 400.)
v.W_n_vap = Range(1., 0., 2.) # moles of water vapor

vsc=VSC(v,s, model3)
vsc.solve(verbosity=1)

0.6079035669169227
0.38267727987803424
0.12068145433502972
0.12068145433502972
0.09566348468144975
0.09566348468144975
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.09488835758089913
0.09034001876457592
0.09034001876457592
0.09034001876457592
0.07785993231196807
0.07785993231196807
0.06998075159133198
0.06998075159133198
0.02506125961643391
0.006596757073373121
0.0025185389699044223
9.762275165274876e-05
4.4128965794413866e-05
4.038416547024041e-08
4.038416547024041e-08
2.4609258425528205e-08
2.669105787540159e-13
2.669105787540159e-13
2.669105787540159e-13
2.669105787540159e-13
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
2.7211155009691965e-23


In [14]:
vsc.rdf

,vector1
,1
v_w_vap,0.000004
W_n_vap,0.228854


In [15]:
vsc.rdf.values # raw numbers
# only 4e-6 m3 of liquid water is lost to the vapor phase compared to the initial
# 1e-3 m3 of liquid water.

array([[4.22513566e-06],
       [2.28853665e-01]])

In [16]:
vsc.vdf

,vector1
,1
T,365.737128
W_n_vap,0.228854
